In [10]:
import cdsapi
import calendar
from subprocess import call


def idmDownloader(task_url, folder_path, file_name):
    """
    IDM下载器
    :param task_url: 下载任务地址
    :param folder_path: 存放文件夹
    :param file_name: 文件名
    :return:
    """
    # IDM安装目录
    idm_engine = "C:\\Program Files (x86)\\Internet Download Manager\\IDMan.exe"
    # 将任务添加至队列
    call([idm_engine, '/d', task_url, '/p', folder_path, '/f', file_name, '/a'])
    # 开始任务队列
    call([idm_engine, '/s'])


if __name__ == '__main__':
    c = cdsapi.Client()  # 创建用户

    # 数据信息字典
    dic = {
        'product_type': 'reanalysis',  # 产品类型
        'format': 'netcdf',  # 数据格式
        'expver': '1',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            'snow_depth', 'soil_temperature_level_1', 'soil_temperature_level_2',
            'soil_temperature_level_3', 'soil_temperature_level_4', 'sub_surface_runoff',
            'surface_net_solar_radiation', 'surface_pressure', 'surface_runoff','total_precipitation', 'volumetric_soil_water_layer_1',
            'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4'],  # 变量名称
        'year': '',  # 年，设为空
        'month': '',  # 月，设为空
        'day': [],  # 日，设为空
        'time': [  # 小时
            '00:00', '01:00', '02:00', '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00', '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00', '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00', '21:00', '22:00', '23:00'],
        'area': [
            31.60531, 121.22500, 31.60530,
            121.22501
        ],#经纬度顺序：北、西、南、东
    }

    # 通过循环批量下载1979年到2020年所有月份数据
    for y in range(2023, 2024):  # 遍历年
        for m in range(6, 7):  # 遍历月
            day_num = calendar.monthrange(y, m)[1]  # 根据年月，获取当月日数
            # 将年、月、日更新至字典中
            dic['year'] = str(y)
            dic['month'] = str(m).zfill(2)
            dic['day'] = [str(d).zfill(2) for d in range(1, day_num + 1)]

            r = c.retrieve('reanalysis-era5-single-levels', dic, )  # 文件下载器
            url = r.location  # 获取文件下载地址
            path = 'C:\\Users\\mz253\\Desktop\\ERA5'  # 存放文件夹C:\Users\mz253\Desktop
            filename = str(y) + str(m).zfill(2) + '.nc'  # 文件名
            idmDownloader(url, path, filename)  # 添加进IDM中下载


2023-08-24 14:13:32,694 INFO Welcome to the CDS
2023-08-24 14:13:32,695 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-08-24 14:13:33,022 INFO Request is queued
2023-08-24 14:13:34,315 INFO Request is running
2023-08-24 14:21:57,291 INFO Request is completed


In [9]:
import xarray as xr
import pandas as pd
import os

# 定义存放 nc 文件的文件夹路径
folder_path = 'C:\\Users\\mz253\\Desktop\\ERA5'

# 获取文件夹中所有的 .nc 文件
nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]

# 创建一个空的 DataFrame 用于存储数据
df_all = pd.DataFrame()

# 遍历每个 .nc 文件并转换为 DataFrame
for nc_file in nc_files:
    file_path = os.path.join(folder_path, nc_file)
    ds = xr.open_dataset(file_path)
    df = ds.to_dataframe().reset_index()
    df_all = pd.concat([df_all, df])

# 将 DataFrame 存储为一个 .csv 文件
csv_file_path = 'C:\\Users\\mz253\\Desktop\\ERA5_2023_qiputang.csv'
df_all.to_csv(csv_file_path, index=False)


In [9]:
import xarray as xr
import pandas as pd
import os

# 定义存放 .nc 文件的文件夹路径
folder_path = 'C:\\Users\\mz253\\Desktop\\ERA5\\2m_temperature'

# 获取文件夹中所有的 .nc 文件
nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]

# 遍历每个 .nc 文件并转换为 Excel 文件
for nc_file in nc_files:
    file_path = os.path.join(folder_path, nc_file)
    ds = xr.open_dataset(file_path)
    df = ds.to_dataframe().reset_index()

    # 提取文件名（不包含扩展名）作为 Excel 文件名
    excel_file_name = os.path.splitext(nc_file)[0] + '.xlsx'

    # 将 DataFrame 存储为一个 Excel 文件
    excel_file_path = os.path.join(folder_path, excel_file_name)
    df.to_excel(excel_file_path, index=False)
